### Extract Data from PDF Files with Python

In [1]:
# !pip install PDFPlumber
import pdfplumber
import pandas as pd
import os

Ejemplo de Conexion con Pyodbc

In [16]:
import pyodbc

server = 'ASUSTUF\SQL22'
database = 'AdventureWorksDW2019'
# username = 'nombre_usuario'
# password = 'contraseña'

try:
    connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                                f'SERVER={server};'
                                f'DATABASE={database};'
                                f'Trusted_Connection=yes;')
    print('Conexión exitosa')
except pyodbc.Error as e:
    print(f'Error al conectar a la base de datos: {str(e)}')

Conexión exitosa


In [4]:
query = "SELECT * FROM vFactFinance"
df_query = pd.read_sql(query, connection)
df_query.head()

C:\Users\rfigu\AppData\Local\Temp\ipykernel_12132\812222591.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_query = pd.read_sql(query, connection)


,FinanceKey,Fecha,Organizacion,Gerencia,Escenario,Descripcion Cuenta,Monto,TotalYTD
0,26702,2012-12-28,Australia,Corporate,Actual,Salaries,19305.12,1392505.88
1,26703,2012-12-28,Australia,Corporate,Actual,Payroll Taxes,1783.74,1392505.88
2,26704,2012-12-28,Australia,Corporate,Actual,Employee Benefits,1188.99,1392505.88
3,26705,2012-12-28,Australia,Corporate,Actual,Travel Transportation,190.73,1392505.88
4,26706,2012-12-28,Australia,Corporate,Actual,Travel Lodging,173.18,1392505.88


Insertar datos mendiate el modulo Cursor

In [28]:
# Insert data to a SQL Server table using Pyodbc librery with handle errors

# ------ Metodo manual -------
# Create a cursor from the connection
cursor = connection.cursor()
try:
   cursor.execute("""INSERT INTO FortalezaTemp VALUES (11353, 'Cuota de Prueba', 'Unidad', 2006, 1450, 10, 1100, 350, 125,  '001-001-0007592', '14-04-2023');""")
   connection.commit()
   print('Fila insertada')
except Exception as e:
   print(f"Error al insertar: {e}")

Fila insertada


In [56]:
df_query = pd.read_sql('SELECT * FROM FortalezaTemp', connection)
df_query.head()

C:\Users\rfigu\AppData\Local\Temp\ipykernel_12132\2171453995.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_query = pd.read_sql('SELECT * FROM FortalezaTemp', connection)


,Codigo,Descripcion,Unidad_medida,Cantidad,Precio_Unitario,Descuento,Exentas,Monto_5,Monto_10,NroFactura,FechaEmision
0,ESPERA20EX,Espera - ESPERA_EXENTA Oct/22 22:240 - Cto.: 1...,UNI,1.0,1088000.0,0.0,1088000.0,0.0,0.0,001-001-0036986,2022-11-10
1,ESPERA20IM,Gastos Administrativos,UNI,1.0,272000.0,0.0,0.0,0.0,272000.0,001-001-0036986,2022-11-10
2,ESPERA20EX,Espera - ESPERA_EXENTA Nov/22 23:240 - Cto.:\n...,UNI,1.0,1088000.0,0.0,1088000.0,0.0,0.0,001-001-0069238,2023-01-30
3,ESPERA20IM,Gastos Administrativos,UNI,1.0,272000.0,0.0,0.0,0.0,272000.0,001-001-0069238,2023-01-30
4,CUOTAEXTIM,EXTRA.ESPERA_GRAVADA Dic/22 5:20 - Cto.: 13467,UNI,1.0,680000.0,0.0,0.0,0.0,680000.0,001-001-0069237,2023-01-30


In [8]:
cursor.close()
connection.close()

Extraer datos de la tabla de la factura PDF

In [ ]:
root_dir = os.getcwd()

print("Ruta base archivos: {}".format(root_dir)) 

df_archivos = []
ciclo = 0

for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith('.pdf'):
            file_paths = os.path.join(dirpath, filename)
            # Se utiliza pdfPlumber para leer los datos de la tabla factura
            with pdfplumber.open(file_paths) as temp:
                print(file_paths)
                ciclo += 1
                    # Nombre de pdf dinamico
                print(f"pdf_{ciclo}")
                first_page = temp.pages[0]
                # print(first_page.extract_table())

In [9]:
diccColumnas = ('Codigo', 'Descripcion', 'Unidad_medida', 'Cantidad', 'Precio_Unitario', 'Descuento', 'Exentas', 'Monto_5', 'Monto_10', 'NroFactura', 'FechaEmision')
diccColumnas

('Codigo',
 'Descripcion',
 'Unidad_medida',
 'Cantidad',
 'Precio_Unitario',
 'Descuento',
 'Exentas',
 'Monto_5',
 'Monto_10',
 'NroFactura',
 'FechaEmision')

In [50]:
root_dir = os.getcwd()

df_archivos = []
ciclo = 0

for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith('.pdf'):
            file_paths = os.path.join(dirpath, filename)
            # Se utiliza pdfPlumber para leer los datos de la tabla factura
            with pdfplumber.open(file_paths) as temp:
                print(file_paths)
                ciclo += 1
                    # Nombre de pdf dinamico
                print(f"pdf_{ciclo}")
                first_page = temp.pages[0]
                        # print(first_page.extract_table())
                dfnames = first_page.extract_table()
                df_t = pd.DataFrame(dfnames)
                headers = df_t.iloc[0].values
                df_t.columns = headers
                df_t.drop(index=0, axis=0, inplace=True)
                df_t = df_t.dropna()

                    # Se agrega el Nro de Factura al Df
                texto = first_page.extract_text()
                nroFact  = texto[0:15]
                print("Nro Factura: {}".format(nroFact)) 
                df_t['NroFactura']=nroFact

                # #Se agrega la fecha y Hora impresa en la factura
                lines = texto.split('\n')
                filtered_lines = [line for line in lines if 'Fecha y hora' in line]
                # print('\n'.join(filtered_lines))
                str(filtered_lines)
                fechafact =str(filtered_lines)[27:37]
                print("Fecha Emision: {}".format(fechafact))
                df_t['FechaEmision']=fechafact
                df_archivos.append(df_t)
                
            df = pd.concat(df_archivos, ignore_index=True)
            df.columns = diccColumnas


c:\Users\rfigu\Documents\Learn\LeerPDF\invoice\29_Octubre_2023.pdf
pdf_1
Nro Factura: 001-001-0036986
Fecha Emision: 10-11-2022
c:\Users\rfigu\Documents\Learn\LeerPDF\invoice\30_Diciembre_2022.pdf
pdf_2
Nro Factura: 001-001-0069238
Fecha Emision: 30-01-2023
c:\Users\rfigu\Documents\Learn\LeerPDF\invoice\31_DicExtraordinario_2022.pdf
pdf_3
Nro Factura: 001-001-0069237
Fecha Emision: 30-01-2023
c:\Users\rfigu\Documents\Learn\LeerPDF\invoice\32_Enero_2023.pdf
pdf_4
Nro Factura: 001-001-0092791
Fecha Emision: 01-04-2023
c:\Users\rfigu\Documents\Learn\LeerPDF\invoice\33_Febrero_2023.pdf
pdf_5
Nro Factura: 001-001-0098732
Fecha Emision: 19-04-2023
c:\Users\rfigu\Documents\Learn\LeerPDF\invoice\34_Mayo_2023.pdf
pdf_6
Nro Factura: 001-001-0106414
Fecha Emision: 10-05-2023
c:\Users\rfigu\Documents\Learn\LeerPDF\invoice\35_Junio_2023.pdf
pdf_7
Nro Factura: 001-001-0118558
Fecha Emision: 13-06-2023
c:\Users\rfigu\Documents\Learn\LeerPDF\invoice\36_Julio_2023.pdf
pdf_8
Nro Factura: 001-001-0132813

In [51]:
df.head()

,Codigo,Descripcion,Unidad_medida,Cantidad,Precio_Unitario,Descuento,Exentas,Monto_5,Monto_10,NroFactura,FechaEmision
0,ESPERA20EX,Espera - ESPERA_EXENTA Oct/22 22:240 - Cto.: 1...,UNI,1.00,1.088.000.0,0.00,1.088.000.0,,,001-001-0036986,10-11-2022
1,ESPERA20IM,Gastos Administrativos,UNI,1.00,272.000.0,0.00,,,272.000.0,001-001-0036986,10-11-2022
2,ESPERA20EX,Espera - ESPERA_EXENTA Nov/22 23:240 - Cto.:\n...,UNI,1.00,1.088.000.0,0.00,1.088.000.0,,,001-001-0069238,30-01-2023
3,ESPERA20IM,Gastos Administrativos,UNI,1.00,272.000.0,0.00,,,272.000.0,001-001-0069238,30-01-2023
4,CUOTAEXTIM,EXTRA.ESPERA_GRAVADA Dic/22 5:20 - Cto.: 13467,UNI,1.00,680.000.0,0.00,,,680.000.0,001-001-0069237,30-01-2023


In [52]:
# Removemos los separadores de miles y el ultimo punto decimal para evitar montos incorrectos
columnasMontos = ['Precio_Unitario', 'Exentas', 'Monto_5', 'Monto_10']
df[columnasMontos] = df[columnasMontos].applymap(lambda x: x.replace('.', ''))
# recotamos 1 caracter a las columnas con valores en moneda
df[columnasMontos] = df[columnasMontos].applymap(lambda x: x[:-1])
df.head(2)

,Codigo,Descripcion,Unidad_medida,Cantidad,Precio_Unitario,Descuento,Exentas,Monto_5,Monto_10,NroFactura,FechaEmision
0,ESPERA20EX,Espera - ESPERA_EXENTA Oct/22 22:240 - Cto.: 1...,UNI,1.00,1088000,0.00,1088000,,,001-001-0036986,10-11-2022
1,ESPERA20IM,Gastos Administrativos,UNI,1.00,272000,0.00,,,272000,001-001-0036986,10-11-2022


In [53]:
# Se extrae los 3 caracteres para las columnas Cuantitativas
colCantidades = ['Cantidad', 'Descuento']
df[colCantidades] = df[colCantidades].applymap(lambda x: x[:-3])
df.head(2)
df

,Codigo,Descripcion,Unidad_medida,Cantidad,Precio_Unitario,Descuento,Exentas,Monto_5,Monto_10,NroFactura,FechaEmision
0,ESPERA20EX,Espera - ESPERA_EXENTA Oct/22 22:240 - Cto.: 1...,UNI,1,1088000,0,1088000,,,001-001-0036986,10-11-2022
1,ESPERA20IM,Gastos Administrativos,UNI,1,272000,0,,,272000,001-001-0036986,10-11-2022
2,ESPERA20EX,Espera - ESPERA_EXENTA Nov/22 23:240 - Cto.:\n...,UNI,1,1088000,0,1088000,,,001-001-0069238,30-01-2023
3,ESPERA20IM,Gastos Administrativos,UNI,1,272000,0,,,272000,001-001-0069238,30-01-2023
4,CUOTAEXTIM,EXTRA.ESPERA_GRAVADA Dic/22 5:20 - Cto.: 13467,UNI,1,680000,0,,,680000,001-001-0069237,30-01-2023
5,ESPERA20EX,Espera - ESPERA_EXENTA Ene/23 25:240 - Cto.:\n...,UNI,1,1088000,0,1088000,,,001-001-0092791,01-04-2023
6,ESPERA20IM,Gastos Administrativos,UNI,1,272000,0,,,272000,001-001-0092791,01-04-2023
7,ESPERA20EX,Espera - ESPERA_EXENTA Feb/23 26:240 - Cto.:\n...,UNI,1,1088000,0,1088000,,,001-001-0098732,19-04-2023
8,ESPERA20IM,Gastos Administrativos,UNI,1,272000,0,,,272000,001-001-0098732,19-04-2023
9,ESPERA20EX,Espera - ESPERA_EXENTA May/23 27:240 - Cto.:\n...,UNI,1,1088000,0,1088000,,,001-001-0106414,10-05-2023


Metodo 1, insertando con Cursor de Pyodbc

In [55]:
for index, row in df.iterrows():
   cursor.execute("INSERT INTO FortalezaTemp (Codigo,Descripcion,Unidad_medida,Cantidad, Precio_Unitario,Descuento,Exentas, Monto_5, Monto_10, NroFactura, FechaEmision) values(?,?,?,?,?,?,?,?,?,?,?)", row.Codigo, row.Descripcion, row.Unidad_medida, row.Cantidad, row.Precio_Unitario, row.Descuento, row.Exentas,row.Monto_5, row.Monto_10, row.NroFactura, row.FechaEmision)
connection.commit()

Metodo 2 Insert con libreria Pandas

### Bloque para leer la factura electronica version 1
Comprende de Marzo de 2022 a Agosto 2022

In [6]:
rutaEspec = r"C:\\Users\\rfigu\\Pictures\\Fortaleza\\facturaElectronicasV1"

df_archivos_v1 = []
ciclo = 0

for dirpath, dirnames, filenames in os.walk(rutaEspec):
    for filename in filenames:
        if filename.endswith('.pdf'):
            file_paths = os.path.join(dirpath, filename)
            # Se utiliza pdfPlumber para leer los datos de la tabla factura
            with pdfplumber.open(file_paths) as temp:
                print(file_paths)
                ciclo += 1
                    # Nombre de pdf dinamico
                print(f"pdf_{ciclo}")
                first_page = temp.pages[0]
                        # print(first_page.extract_table())
                dfnames = first_page.extract_table()
                df_t = pd.DataFrame(dfnames)
                headers = df_t.iloc[0].values
                df_t.columns = headers
                df_t.drop(index=0, axis=0, inplace=True)
                df_t = df_t.dropna()

                    # Se agrega el Nro de Factura al Df # factura version 1
                texto = first_page.extract_text()
                nroFact  = texto[327:342]
                print("Nro Factura: {}".format(nroFact)) 
                df_t['NroFactura']=nroFact

                #   #Se agrega la fecha y Hora impresa en la factura version 1
                lines = texto.split('\n')
                filtered_lines = [line for line in lines if 'Fecha de Emisión:' in line]
                fechafact =str(filtered_lines)[20:30]
                print("Fecha Emision: {}".format(fechafact))
                df_t['FechaEmision']=fechafact
                df_archivos_v1.append(df_t)
                            # if ciclo == 2:
                            #     break    # break here
                
            df_v1 = pd.concat(df_archivos_v1, ignore_index=True)
            df_v1.columns = diccColumnas

C:\\Users\\rfigu\\Pictures\\Fortaleza\\facturaElectronicasV1\22_Marzo2022.pdf
pdf_1
Nro Factura: 001-002-0010793
Fecha Emision: 10/03/2022
C:\\Users\\rfigu\\Pictures\\Fortaleza\\facturaElectronicasV1\23_Abril2022.pdf
pdf_2
Nro Factura: 001-001-0010087
Fecha Emision: 11/04/2022
C:\\Users\\rfigu\\Pictures\\Fortaleza\\facturaElectronicasV1\24_Mayo_2022.pdf
pdf_3
Nro Factura: 001-004-0008842
Fecha Emision: 11/05/2022
C:\\Users\\rfigu\\Pictures\\Fortaleza\\facturaElectronicasV1\25_Junio_2022.pdf
pdf_4
Nro Factura: 001-003-0003504
Fecha Emision: 10/06/2022
C:\\Users\\rfigu\\Pictures\\Fortaleza\\facturaElectronicasV1\26_Julio_2022.pdf
pdf_5
Nro Factura: 001-003-0005000
Fecha Emision: 12/07/2022
C:\\Users\\rfigu\\Pictures\\Fortaleza\\facturaElectronicasV1\26_Junio_2022_Extra.pdf
pdf_6
Nro Factura: 001-001-0014565
Fecha Emision: 13/06/2022
C:\\Users\\rfigu\\Pictures\\Fortaleza\\facturaElectronicasV1\27_Agosto_2022.pdf
pdf_7
Nro Factura: 001-001-0018137
Fecha Emision: 11/08/2022


In [7]:
df_v1.head()

,Codigo,Descripcion,Unidad de nmedida,Cantidad,Precio Unitario,Descuento,Exentas,Monto 5%,Monto 10%,NroFactura,FechaEmision
0,ESPERA20IM Ga\nESPERA20EX Es,stos Administativos\npera 03/22 : 15 - Cto.: 1...,Unidad\nUnidad,1.000000\n1.000000,247.000\n988.000,0\n0,0\n988.000,0\n0,247.000\n0,001-002-0010793,10/03/2022
1,ESPERA20IM Ga\nESPERA20EX Es,stos Administativos\npera 04/22 : 16 - Cto.: 1...,Unidad\nUnidad,1.000000\n1.000000,247.000\n988.000,0\n0,0\n988.000,0\n0,247.000\n0,001-001-0010087,11/04/2022
2,ESPERA20IM Ga\nESPERA20EX Es,stos Administativos\npera 05/22 : 17 - Cto.: 1...,Unidad\nUnidad,1.0000\n1.0000,258.00\n1.032.00,0 0\n0 0,0\n1.032.000,0\n0,258.000\n0,001-004-0008842,11/05/2022
3,ESPERA20IM Ga\nESPERA20EX Es,stos Administativos\npera 06/22 : 18 - Cto.: 1...,Unidad\nUnidad,1.0000\n1.0000,258.00\n1.032.00,0 0\n0 0,0\n1.032.000,0\n0,258.000\n0,001-003-0003504,10/06/2022
4,ESPERA20IM Ga\nESPERA20EX Es,stos Administativos\npera 07/22 : 19 - Cto.: 1...,Unidad\nUnidad,1.0000\n1.0000,258.00\n1.032.00,0 0\n0 0,0\n1.032.000,0\n0,258.000\n0,001-003-0005000,12/07/2022
5,CUOTAEXTIM Cu,ota Ext. 06/22 : 4 - Cto.: 13467,Unidad,1.0000,645.00,0 0,0,0,645.000,001-001-0014565,13/06/2022
6,ESPERA20IM Ga\nESPERA20EX Es,stos Administativos\npera 08/22 : 20 - Cto.: 1...,Unidad\nUnidad,1.0000\n1.0000,258.00\n1.032.00,0 0\n0 0,0\n1.032.000,0\n0,258.000\n0,001-001-0018137,11/08/2022


In [ ]:
# Nro de factura en la factra v2
texto = first_page.extract_text()
nroFact  = texto[327:342]
print("Nro Factura: {}".format(nroFact)) 

In [ ]:
lines = texto.split('\n')
filtered_lines = [line for line in lines if 'Fecha de Emisión:' in line]
fechafact =str(filtered_lines)[20:30]
print("Fecha Emision: {}".format(fechafact))